### 1. Install and Import Dependecies

In [1]:
!pip3 install torch torchvision torchaudio


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!pip install transformers requests beautifulsoup4 pandas numpy


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModelForMaskedLM
import torch
import requests
from bs4 import BeautifulSoup
import re

### 2. Instantiate Model

In [4]:
tokenizer = AutoTokenizer.from_pretrained("google/muril-base-cased")
model = AutoModelForSequenceClassification.from_pretrained("google/muril-base-cased")

tokenizer_config.json:   0%|          | 0.00/206 [00:00<?, ?B/s]

c:\Python312\Lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\hpind\.cache\huggingface\hub\models--google--muril-base-cased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/3.16M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/113 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/953M [00:00<?, ?B/s]

OSError: [Errno 28] No space left on device

### 3. Encode and Calculate Sentiment

In [ ]:
tokens = tokenizer.encode('I hated this, absolutely the worse', return_tensors='pt')

In [ ]:
tokens

tensor([[   104,    148, 154519,   1222,    119,  35880,   1108,  40253,    105]])

In [ ]:
tokenizer.decode(tokens[0])

'[CLS] I hated this, absolutely the worse [SEP]'

In [ ]:
tokens2 = tokenizer.encode('Ye bohot bura hua', return_tensors='pt')

In [ ]:
tokens2

tensor([[   104, 144380, 190739,  81025,  67440,   7029,    105]])

In [ ]:
tokens3 = tokenizer.encode('Mala khup anand zala', return_tensors='pt')
tokens3

tensor([[  104, 90453,   182,  9623,  2208, 25838, 64126,   105]])

In [ ]:
result1 = model(tokens)

In [ ]:
result1.logits

tensor([[-0.0009, -0.0059]], grad_fn=<AddmmBackward0>)

In [ ]:
torch.argmax(result1.logits)

tensor(0)

In [ ]:
tokens4 = tokenizer.encode('Meh it was good but could have been better. Great', return_tensors='pt')
tokens4

tensor([[  104, 14375,  1635,  1197,  1140,  2525,  1363,  2495,  1226,  1352,
          5142,   121,  5846,   105]])

In [ ]:
result2 = model(tokens4)

In [ ]:
result2.logits

tensor([[-0.0010, -0.0061]], grad_fn=<AddmmBackward0>)

In [ ]:
torch.argmax(result2.logits)

tensor(0)

In [ ]:
result3 = model(tokens3)
result3.logits

tensor([[-0.0009, -0.0059]], grad_fn=<AddmmBackward0>)

In [ ]:
torch.argmax(result3.logits)

tensor(0)

### 4. Collect News

In [ ]:
r = requests.get('https://www.abplive.com/news')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*news.*')
results = soup.find_all('a', {'class':regex})
news = [result.text for result in results]

In [ ]:
news

['इंडिया',
 'विश्व',
 'न्यूज़',
 'विश्व',
 'इंडिया',
 'अपराध',
 'यूटिलिटी न्यूज़',
 'विश्वक्या भारत-बांग्लादेश के बीच कभी सुधरेंगे रिश्ते? मोहम्मद यूनुस ने कर दिया बड़ा खुलासा',
 'विश्वजर्मनी में आतंकी हमले की आशंका! ड्राइवर ने भीड़ पर चढ़ाई कार, एक की मौत, हिरासत में संदिग्ध',
 'विश्वबांग्लादेश में उथल-पुथल मचाने की कौन रच रहा साजिश? मोहम्मद यूनुस ने कर दिया बड़ा खुलासा',
 'इंडियाबेटी की खैरियत जानने पिता पहुंचा HC, सरकार बोली- शहजादी खान को पिछले महीने ही दुबई में दी जा चुकी है फांसी',
 "इंडियारश्मिका मंदाना ने किया कन्नड़ भाषा का अपमान! कांग्रेस विधायक बोले- 'उन्हें सबक सिखाना चाहिए'",
 '\n\nविश्व\n\nन्यूक्लियर पावर बनने ही वाला था ताइवान, इस एजेंट ने अमेरिका को लीक कर दिया था सीक्रेट प्लान\n\t\t\t\t\t\t\t\t\t\n',
 '\n\nविश्व\n\nअमेरिका के कैरोलिना के जंगल में दहक रही आग, डोनाल्ड ट्रंप बीच के किनारे खेलते नजर आए गोल्फ\n\t\t\t\t\t\t\t\t\t\n',
 "\n\nक्रिकेट\n\nरोहित शर्मा को 'मोटा' कहने पर भारी बवाल, BJP बोली- राहुल गांधी अनफिट, हमें भारतीय कप्तान पर गर्व\n\t\t\t\t\t\t\t\t\t\n",
 "\n\

 ### 5. Load News into DataFrame and Score

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.DataFrame(np.array(news), columns=['news'])

In [ ]:
df.tail()

,news
57,क्रिकेटकब और कहां खेले जाएंगे 2025 चैंपियंस ट्...
58,विश्वबांग्लादेश में उथल-पुथल मचाने की कौन रच र...
59,ट्रेंडिंगलगता है पहली बार पाला पड़ा है...शॉपिं...
60,Blogअलग-अलग राहों पर बिहार में चल पड़े राजद और ...
61,हेल्थआसपास मौजूद इन चीजों से बहरे हो रहे हैं आ...


In [ ]:
df['news'].iloc[45]

'\n\nविश्व\n\n\n\n\n\n            जर्मनी में आतंकी हमले की आशंका! ड्राइवर ने भीड़ पर चढ़ाई कार, एक की मौत, हिरासत में संदिग्ध\n        \n'

In [ ]:
def sentiment_score(news):
    tokens = tokenizer.encode(news, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [ ]:
sentiment_score(df['news'].iloc[9])

1

In [ ]:
df['sentiment'] = df['news'].apply(lambda x: sentiment_score(x[:512]))

In [ ]:
df

,news,sentiment
0,इंडिया,1
1,विश्व,1
2,न्यूज़,1
3,विश्व,1
4,इंडिया,1
...,...,...
57,क्रिकेटकब और कहां खेले जाएंगे 2025 चैंपियंस ट्...,1
58,विश्वबांग्लादेश में उथल-पुथल मचाने की कौन रच र...,1
59,ट्रेंडिंगलगता है पहली बार पाला पड़ा है...शॉपिं...,1
60,Blogअलग-अलग राहों पर बिहार में चल पड़े राजद और ...,1
